# Nappe de Morlces, Switzerland

Notebook crated by [Bane Sullivan](https://banesullivan.com)


Download and extract that data from the archive: https://doi.org/10.6084/m9.figshare.c.4130759

> Dataset representing a 3D geological reconstruction of the a section of the Nappe de Morlces, Switzerland. Associated with "A 3D geological model of a structurally complex Alpine region as a basis for interdisciplinary research" (Thornton et al.)

> Thornton, J. M., Mariethoz, G., & Brunner, P. (2018). A 3D geological model of a structurally complex Alpine region as a basis for interdisciplinary research. https://doi.org/10.6084/m9.figshare.6507437.v1

In [ ]:
import numpy as np
import PVGeo
import pyvista as pv
import glob
import os
from matplotlib.colors import ListedColormap

In [ ]:
import sys
import pandas as pd
#from matplotlib.colors import ListedColormap

In [ ]:
pv.rcParams["colorbar_orientation"] = "vertical"
pv.rcParams["colorbar_vertical"]["height"] = 0.9

In [ ]:
# colormap
colors = {
    "10_basement": "#d1d1d1",
    "11_a": "#8b5a2b",
    "12_i1i": "#cd853f",
    "13_i1s": "#ee9a49",
    "14_i2": "#deb887",
    "15_i3_4": "#6e8b3d",
    "16_i5": "#5f9ea0",
    "17_i6_8": "#7ac5cd",
    "18_i8c2": "#8fbc8f",
    "19_c2k": "#9acd32",
    "20_c3": "#cd9b1d",
    "21_c4": "#9acd32",
    "22_c4_5": "#548b54",
    "23_c6_8": "#ff6347",
    "24_e5": "#f08080",
    "25_e6c": "#f4a460",
    "26_e6k": "#eedc82",
    "27_e6o1": "#ffff00",
}
cmap = ListedColormap(colors.values())

annotations = dict([(int(k[:2]), k) for k in colors.keys()])

dargs = dict(cmap=cmap, clim=[9.5, 27.5], n_colors=len(colors),
             annotations=annotations, 
             scalar_bar_args=dict(n_labels=0, position_x=0.15, 
                                  interactive=True, title_font_size=12))

In [ ]:
# Only set to True for co-dev of ela from this use case:
ela_from_source = False
ela_from_source = True

In [ ]:
if ela_from_source:
    if ('ELA_SRC' in os.environ):
        root_src_dir = os.environ['ELA_SRC']
    elif sys.platform == 'win32':
        root_src_dir = r'C:\src\github_jm\pyela'
    else:
        username = os.environ['USER']
        root_src_dir = os.path.join('/home', username, 'src/ela/pyela')
    pkg_src_dir = root_src_dir
    sys.path.insert(0, pkg_src_dir)

from ela.textproc import *
from ela.utils import *
from ela.classification import *
from ela.visual import *
from ela.spatial import SliceOperation

In [ ]:
from ela.visual import discrete_classes_colormap
# from ela.VisualizeDataProcess import VisualizeDataProcess

from pyvista_sample.VisualizeDataProcess import VisualizeDataProcess


## Surfaces

Load surfaces representing the lithology regions of the 3D model

In [ ]:
# Load poly data surface
# path = "/home/per202/data/tmp/Thornton/Surfaces for ParaView/"
# files = [f for f in glob.glob(path + "*.vtp")]
import pickle

data_path = '/home/per202/data/Lithology/Canberra/out'
fp = os.path.join(data_path, '3d_primary_litho.pkl')
with open(fp, 'rb') as handle:
    lithology_3d_array = pickle.load(handle, encoding='latin1')

    

In [ ]:
lithology_3d_array[102,102,102]
    

In [ ]:
dp = VisualizeDataProcess()

In [ ]:
layer = dp.build_layer_mesh(lithology_3d_array, height_adjustment_factor=10, layer_from=0, layer_to=6)

In [ ]:
print(layer.contour.__doc__)

In [ ]:
# layer.contour(isosurfaces=np.arange(0,6) + 0.5)
iso = layer.contour(isosurfaces=6)

In [ ]:
#set up the style of scalar bar and annotations
annotations = {
    00.0: 'shale',
    01.0: 'clay',
    02.0: 'granite',
    03.0: 'soil',
    04.0: 'sand',
    05.0: 'porphyry',
    06.0: 'siltstone',
    07.0: 'dacite',
    08.0: 'rhyodacite',
    09.0: 'gravel',
    10.0: 'limestone',
    11.0: 'sandstone',
    12.0: 'slate',
    13.0: 'mudstone',
    14.0: 'rock',
    15.0: 'ignimbrite',
    16.0: 'tuff'
}

In [ ]:
plotter = pv.Plotter()

In [ ]:
# plotter.add_mesh(layer, scalars="Lithology", n_colors=len(annotations), clim=[0, len(annotations) - 1],
#                  show_scalar_bar=False)
plotter.add_mesh(iso, scalars="Lithology", n_colors=len(annotations), clim=[0, len(annotations) - 1],
                 show_scalar_bar=False)

In [ ]:
plotter.show_axes()


In [ ]:
plotter.show(title="Canberra",use_panel=True,auto_close=False)

In [ ]:
read = lambda f: (os.path.basename(f)[:-4], pv.read(f))
surfaces = pv.MultiBlock(dict([read(f) for f in files]))

surfaces

In [ ]:
p = pv.Plotter(notebook=1)
for i, surf in enumerate(surfaces):
    name = surfaces.get_block_name(i)
    p.add_mesh(surf, color=colors[name])
p.show()

## 3D Model

Load a 3D model and do some cool stuff!

In [ ]:
# path = "Thornton_et_al/SGeMS voxel format/"
# files = [f for f in glob.glob(path + "*.SGEMS")]

def read_sgems(filename, extent):
    """String filename and (nx,ny,nz) extent"""
    extent = np.append(np.array(extent), 1)
    table = PVGeo.gslib.GSLibReader().apply(filename)
    grid = PVGeo.grids.TableToTimeGrid(extent=extent).apply(table)
    grid.origin = (570500.0, 112400.0, 0.0)
    return grid

grid_50 = read_sgems('/home/per202/data/tmp/Thornton/SGeMS voxel format/50m.SGEMS', (192, 268, 65))
grid_50.spacing = (50, 50, 50)
# remove invalid regions of model (zeros)
model = grid_50.threshold(0.5)
model

In [ ]:
box = [570500.0+10, 580100.0-10, 
       112400.0+10, 125800.0-10, 
       450.0+10, 3250.0-10]
topo = grid_50.threshold(0.0, 
                         invert=True).extract_surface().clip_box(box, 
                                                                 invert=False)
topo = topo.extract_surface().smooth(500).elevation()
p = pv.Plotter()
p.add_mesh(topo, scalars="Elevation", cmap="terrain")
p.show(use_panel=True, auto_close=False)

In [ ]:
p = pv.Plotter(notebook=True)
p.add_mesh(model, **dargs)
p.show(use_panel=True, auto_close=False)

In [ ]:
p = pv.Plotter(notebook=True)
p.add_mesh(model.ctp().contour(17), **dargs)
p.show(use_panel=True, auto_close=False)

In [ ]:
# This is interactive!!!!

p = pv.Plotter(notebook=0)
# p.add_mesh(topo, opacity=0.75, cmap="gist_earth")
p.add_mesh(model.slice_orthogonal(z=1100), **dargs)


def callback(value):
    index = int(value)
    name = annotations[index]
    p.add_mesh(surfaces[name], color=colors[name], 
               reset_camera=False, name="surf_of_interest")
    return

p.add_slider_widget(callback, [10, 27], pointb=(0.75,0.9))
p.show()